In [ ]:
import datasets

dataset = datasets.load_dataset("lmarena-ai/arena-human-preference-100k")
df = dataset["train"].to_pandas()

# limit to between 3 and 10 turns
df = df[df["turn"].between(3, 10)]

# limit to non-tie votes
df = df[df["winner"].isin(["model_a", "model_b"])]

# limit to English-language conversations
df = df[df["language"] == "English"]

# remove token counts from each turn
def remove_token_counts(row):
    for conversation in ["conversation_a", "conversation_b"]:
        conv_list = row[conversation]
        for turn in conv_list:
            turn.pop("num_tokens")
        row[conversation] = conv_list
    return row

df = df.sample(2000)

df = df.apply(remove_token_counts, axis=1)

In [ ]:
df["text_a"] = df["conversation_a"]
df["text_b"] = df["conversation_b"]
df["preferred_text"] = df["winner"]

def transform_winner(row):
    if row["winner"] == "model_a":
        return "text_a"
    else:
        return "text_b"

df["preferred_text"] = df.apply(transform_winner, axis=1)

df.to_csv("../data/output/arena100k_multiturn_2k_english.csv", index=False)